In [81]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer,VectorIndexer,VectorAssembler,OneHotEncoder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [82]:
spark = SparkSession.builder.appName('CustomerChurnProj').getOrCreate()

In [83]:
data = spark.read.csv('customer_churn.csv',header=True,inferSchema=True)
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [84]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [85]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [86]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [87]:
data.na.drop()

DataFrame[Names: string, Age: double, Total_Purchase: double, Account_Manager: int, Years: double, Num_Sites: double, Onboard_date: timestamp, Location: string, Company: string, Churn: int]

In [88]:
data.na.drop()

DataFrame[Names: string, Age: double, Total_Purchase: double, Account_Manager: int, Years: double, Num_Sites: double, Onboard_date: timestamp, Location: string, Company: string, Churn: int]

In [89]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [90]:
output = assembler.transform(dataset=data)
final_data = output.select('features','churn')

In [91]:
train_data,test_data = final_data.randomSplit(weights=[0.7,0.3],seed=4)

In [92]:
lr = LogisticRegression(labelCol='churn')
fitted_churn_model = lr.fit(train_data)
training_sum = fitted_churn_model.summary

In [93]:
training_sum.predictions.describe().show()

+-------+------------------+-------------------+
|summary|             churn|         prediction|
+-------+------------------+-------------------+
|  count|               619|                619|
|   mean|0.1825525040387722|0.13893376413570274|
| stddev|0.3866116334084656|0.34615711933909693|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



In [94]:
pred_and_labels =  fitted_churn_model.evaluate(test_data)
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[4.58573140941480...|[0.98990662556897...|       0.0|
|[28.0,11128.95,1....|    0|[4.12528488336793...|[0.98409806706040...|       0.0|
|[29.0,9378.24,0.0...|    0|[4.60095750107158...|[0.99005762773941...|       0.0|
|[29.0,10203.18,1....|    0|[3.62336739389151...|[0.97400133272507...|       0.0|
|[29.0,12711.15,0....|    0|[5.30693858194052...|[0.99506737222297...|       0.0|
|[30.0,7960.64,1.0...|    1|[3.10593866437310...|[0.95713704555500...|       0.0|
|[30.0,8677.28,1.0...|    0|[3.80750632045817...|[0.97827880768446...|       0.0|
|[30.0,10183.98,1....|    0|[2.80395015156242...|[0.94288890995776...|       0.0|
|[30.0,10960.52,1....|    0|[2.30160386676627...|[0.90900978345604...|       0.0|
|[31.0,5304.6,0.

In [95]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [96]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [97]:
auc

0.8173460345591492

In [98]:
final_lr_model = lr.fit(final_data)

In [99]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [100]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [101]:
test_new_customers = assembler.transform(new_customers)

In [102]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [103]:
final_results = final_lr_model.transform(test_new_customers)

In [104]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

